## RiboSeq processing pipeline 

#### Steps

This pipe assumes you have the fastq files to process in a dedicated directory within a subdirectory called data.

1) Run Fastqc to check the read length distribution.  Size selection was used during the experiment, expect both 26-34nt for monosome and 54-68nt for disome according to Erica's email.

2) Run cutadapt using the parameters provided by Ezrabio.    "-j 8 -g "^GGG" -a "A{10}" -n 2 -m 15 --max-n=0.1 --discard-casava -o output.fastq.gz input.fastq.gz"

3) Remove reads where the first position quality score is <=10

4) Align reads with Bowtie to non-coding RNA, https://downloads.yeastgenome.org/sequence/S288C_reference/rna/archive/rna_coding_R64-1-1_20110203.fasta.gz reads that align will be discarded.  Allow 1 mismatch in bowtie alignment.

Alignment files (SAM/BAM) provide forward/reverse alignment information is provided by the Sam Flag.  
    0 is read aligned in the Fwd direction
    4 is unaligned
    16 is aligned in the Rvs direction

5) Align the remaining reads with Bowtie to YPS1009 and S288C reference genomes.

6) Run samtools mpileup to generate base counts for all genes in YPS1009 and S288C.





Samples labeled as -RNA are strictly RNA-Seq and will be processed with the normal condor RNA-Seq pipeline.

/mnt/bigdata/linuxhome/mplace/data/Ribo-seq/Leah-12142023/RNA-Seq

~/scripts/Condor-RNA-Seq-Pipeline/rnaSeqCondor_paired.py -e mplace@wisc.edu -f filelist -ref R64

~/scripts/Condor-RNA-Seq-Pipeline/rnaSeqCondor_paired.py -e mplace@wisc.edu -f filelist -ref YPS1009

In [2]:
# import required Python modules
import glob
import itertools
import os
import re
import subprocess
import sys

# set global variables
parentDir = os.getcwd() + "/"
referenceDir = "/mnt/bigdata/linuxhome/mplace/scripts/riboSeqPipeline/reference/"
print(parentDir)

/mnt/bigdata/linuxhome/mplace/data/Ribo-seq/Leah-12142023/


### Create the list of fastq files to process

In [ ]:
# create a list file of fastq files for processing, we are assuming the original fastq files are in a directory called data.
dataDir =  parentDir + "data/"        
with open('inputFastq.txt', 'w') as out:
    for fstq in glob.glob(dataDir + "*.fastq"):
        out.write(fstq + "\n")               

### Write fastqc condor submit files

In [ ]:
# Step 1)
# write the fastqc condor submit file
with open('fastqc.submit', 'w') as submit:
            submit.write( "Universe                 = vanilla\n" )
            submit.write( "Executable               = runFastqc.sh\n")
            submit.write( "Arguments                = $(fastqFile)\n")
            submit.write( "Error                    = fastqc.submit.err\n")
            submit.write( "Log                      = fastqc.submit.log\n")  
            submit.write( "Requirements             = OpSysandVer == \"CentOS7\"\n")
            submit.write( "Queue fastqFile from inputFastq.txt\n" )
submit.close()  

# write shell script to run fastqc
with open('runFastqc.sh', 'w') as out:
    out.write("#!/bin/bash\n")
    out.write("source /opt/bifxapps/miniconda3/etc/profile.d/conda.sh\n")
    out.write("unset PYTHONPATH\n")  
    out.write("conda activate /home/glbrc.org/mplace/.conda/envs/riboSeq\n")
    out.write("fastqc $1\n")
    out.write("conda deactivate")
out.close()

os.chmod('runFastqc.sh', 0o0777)


### Using the GLBRC condor submit node submit the above fastq.submit file

 condor_submit fastqc.submit  

 check that the job is running

 condor_q , you should see something like the following

Schedd: scarcity-submit.glbrc.org : <144.92.98.21:9618?... @ 12/04/23 14:27:03

OWNER  BATCH_NAME    SUBMITTED   DONE   RUN    IDLE  TOTAL JOB_IDS

mplace ID: 814372  12/4  14:25      _      _      2      2 814372.0-1



### Prepare to run Cutadapt 

In [ ]:
# Step 2)
# setup input file for cutadapt
cutadaptOutDir = parentDir + 'cutadapt/'
if os.path.exists(cutadaptOutDir):
    print("Directory exists.")
else:
    os.mkdir(cutadaptOutDir)

with open('inputFastq.txt', 'r') as f, open('cutadaptInput.txt', 'w') as out:
    for fstq in f:
        fstqName = re.sub('.fastq', '-clean.fastq', os.path.basename(fstq.rstrip()))
        fstqOutName = cutadaptOutDir + fstqName
        out.write(f'{fstq.rstrip()} {fstqOutName}\n')
f.close()
out.close()

### Write cutadapt condor submit files

In [ ]:
# write the cutadapt condor submit file
with open('cutadapt.submit', 'w') as submit:
    submit.write( "Universe                 = vanilla\n" )
    submit.write( "Executable               = runCutAdapt.sh\n")
    submit.write( "Arguments                = $(fastqFile) $(outFastq)\n")
    submit.write( "Error                    = cutadapt.submit.err\n")
    submit.write( "Log                      = cutadapt.submit.log\n")  
    submit.write( "Requirements             = OpSysandVer == \"CentOS7\"\n")
    submit.write( "Queue fastqFile, outFastq from cutadaptInput.txt\n" )
submit.close()

# write shell script to run cutadapt
with open('runCutAdapt.sh', 'w') as out:
    out.write("#!/bin/bash\n")
    out.write("source /opt/bifxapps/miniconda3/etc/profile.d/conda.sh\n")
    out.write("unset PYTHONPATH\n")  
    out.write("conda activate /home/glbrc.org/mplace/.conda/envs/riboSeq\n")
    out.write("cutadapt -j 8 -g ^GGG -a A{10} -n 2 -m 15 --max-n=0.1 --discard-casava -o $2 $1\n")
    out.write("conda deactivate")
out.close()

os.chmod('runCutAdapt.sh', 0o0777)

###  on scarcity-submit run the above cutadapt condor files

condor_submit cutadapt.submit

### Next remove reads where the 1st position quality score is low.
### This only removes a handful of reads.

In [ ]:
# Step 3) Remove reads where the first position quality score is <=10

qualScores = ['+', '*', ')','(', "'", '&', '%', '$', '#', '"', '!']
for fastq in glob.glob('cutadapt/*-clean.fastq'):
    name = re.sub('-clean.fastq', '-filt.fastq', fastq)
    with open(fastq, 'r') as f, open(name, 'w') as out:
        for hdr, seq, plus, qual in itertools.zip_longest(*[f]*4):
            firstQual = [*qual][0]
            if not firstQual in qualScores:
                out.write(hdr)
                out.write(seq)
                out.write(plus)
                out.write(qual)
            else:
                print(hdr, '  ', qual)



### Align reads to Non-coding RNA using Bowtie2.  
### Identifies which reads to remove from further consideration.

In [ ]:
# Step 4 ) Align reads with Bowtie2 to non-coding RNA, 
# https://downloads.yeastgenome.org/sequence/S288C_reference/rna/archive/rna_coding_R64-1-1_20110203.fasta.gz 
# reads that align will be discarded.  Allow 1 mismatch in bowtie2 alignment.
# bowtie2 -p 8 --phred33 -N 1 -x $REFERENCE -U $file -S $out.sam 
# -p number of threads
# -N Sets the number of mismatches
# -x The basename of the index for the reference genome
# -U file to align (unpaired)
# -S File to write SAM alignments to

# setup input file for bowtie2 alignment to non-coding RNA
nonCodingOutDir = parentDir + 'alignNonCodingRNA/'
if os.path.exists(nonCodingOutDir):
    print("Directory exists.")
else:
    os.mkdir(nonCodingOutDir)

# get a list of cutadapt cleaned & filtered fastq files for alignment
with open('alignmentInput.txt', 'w') as out:
    for cleanfstq in glob.glob(cutadaptOutDir + '*-filt.fastq'):
        samFile = re.sub('cutadapt', 'alignNonCodingRNA', re.sub('-filt.fastq', '.sam', cleanfstq))        
        out.write(cleanfstq + ' ' + samFile + '\n')
out.close()

### Write Bowtie2 condor submit files

In [ ]:
# write the bowtie2 condor submit file
with open('ncbowtie2.submit', 'w') as submit:
    submit.write( "Universe                 = vanilla\n" )
    submit.write( "Executable               = runncBowtie2.sh\n")
    submit.write( "Arguments                = $(fastqFile) $(sam)\n")
    submit.write( "Error                    = ncbowtie2.submit.err\n")
    submit.write( "Log                      = ncbowtie2.submit.log\n")  
    submit.write( "Requirements             = OpSysandVer == \"CentOS7\"\n")
    submit.write( "Queue fastqFile, sam from alignmentInput.txt\n" )
submit.close()

# write shell script to run cutadapt
with open('runncBowtie2.sh', 'w') as out:
    out.write("#!/bin/bash\n")
    out.write("source /opt/bifxapps/miniconda3/etc/profile.d/conda.sh\n")
    out.write("unset PYTHONPATH\n")  
    out.write("conda activate /home/glbrc.org/mplace/.conda/envs/riboSeq\n")
    out.write("bowtie2 -p 8 --phred33 -N 1 -x /mnt/bigdata/linuxhome/mplace/scripts/riboseq/reference/rna_coding_R64-1-1 -U $1 -S $2\n")
    out.write("conda deactivate")
out.close()

os.chmod('runncBowtie2.sh', 0o0777)

###  on scarcity-submit run the above condor files

condor_submit ncbowtie2.submit

### Remove reads that aligned to Non-coding using samtools view.
#### -F --exclude-flags FLAG ,     Do not output alignments with any bits set in FLAG present in the FLAG field.
#### in this case we exclude the reads which did not align to the Non-coding RNA i.e. the reads we want to analyze later.

In [ ]:
# get a list of reads which aligned to Non-Coding RNA and remove them from the clean.fastq files
#samtools view -F 4 -u  -O SAM -o mapped.sam TestSample1.sam
with open('filterSamInput.txt', 'w') as out:
    for sam in glob.glob(nonCodingOutDir + '*.sam'):
        sampleName = re.sub('.sam', '', os.path.basename(sam))
        outSam = nonCodingOutDir + sampleName + '-unmapped.sam'
        out.write(sam + ' ' + outSam + '\n')
out.close()

### Write the samtools condor submit file to create the clean fastq files for the next step.

In [ ]:
# write the samtools filter UNMAPPED (reads which did not align to Non-Coding RNA) reads condor submit file
# these reads will be aligned to S288C and YPS1009
with open('filter.submit', 'w') as submit:
    submit.write( "Universe                 = vanilla\n" )
    submit.write( "Executable               = runfilter.sh\n")
    submit.write( "Arguments                = $(sam) $(ncsam)\n")
    submit.write( "Error                    = filter.submit.err\n")
    submit.write( "Log                      = filter.submit.log\n")  
    submit.write( "Requirements             = OpSysandVer == \"CentOS7\"\n")
    submit.write( "Queue sam, ncsam from filterSamInput.txt\n" )
submit.close()

# write shell script to run cutadapt
with open('runfilter.sh', 'w') as out:
    out.write("#!/bin/bash\n")
    out.write("source /opt/bifxapps/miniconda3/etc/profile.d/conda.sh\n")
    out.write("unset PYTHONPATH\n")  
    out.write("conda activate /home/glbrc.org/mplace/.conda/envs/riboSeq\n")
    out.write("samtools view -f 4 -u -O SAM -o $2 $1\n")
    out.write("conda deactivate")
out.close()

os.chmod('runfilter.sh', 0o0777)

###  on scarcity-submit run the above condor files

condor_submit filter.submit

### Create the alignment results directories

In [ ]:
# create output files for genome alignments
if not os.path.exists(parentDir + 'alignments'):
    os.mkdir(parentDir + 'alignments')
    os.mkdir(parentDir + 'alignments/S288C')
    os.mkdir(parentDir + 'alignments/YPS1009')

### Create new fastq files with the reads which did not map to the Non-Coding RNA

In [ ]:
# Create new fastq files by filtering for reads that DID NOT ALIGN to the Non-Coding RNA
# First create a new file containing the read names (for reads we want to keep)
# nonCodingOutDir
for unmapped in glob.glob(nonCodingOutDir + '*-unmapped.sam'):
    nameFile = re.sub('-unmapped.sam', '', os.path.basename(unmapped))
    outFile  = parentDir + 'alignments/' + nameFile + '-names.txt'
    with open(unmapped) as f, open(outFile, 'w') as out:
        for line in f:
            name = line.split('\t')[0]
            out.write(f'{name}\n')
    f.close()
    out.close()

In [ ]:

# Use seqtk to subset the unmapped reads for use with bowtie2
for fstq in glob.glob(cutadaptOutDir + '*-filt.fastq'):
    print('processing: ', fstq)
    outFile = parentDir + 'alignments/' +  re.sub('-filt.fastq', '.fastq', os.path.basename(fstq))    # create output file name
    nameLst = parentDir + 'alignments/' + re.sub('-filt.fastq', '-names.txt', os.path.basename(fstq)) 
    cmd = [ 'seqtk', 'subseq', fstq, nameLst ]
    # run command and capture output
    output = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE).communicate()    
    # write results to file
    with open(outFile, 'w') as out:
        out.write(output[0].decode('utf-8'))
    out.close()     

### Set up for alignments to S288C

In [ ]:
# Create input file for alignment to the S288C reference genome.
outputPath = parentDir + 'alignments/S288C/'
with open('refAlignmentS288C_input.txt', 'w') as out:
    for inFastq in glob.glob(parentDir + 'alignments/*.fastq'):
        sampleName = re.sub('.fastq', '.sam', os.path.basename(inFastq))
        out.write(inFastq + ' ' + outputPath + sampleName + ' /home/glbrc.org/mplace/data/reference/S288C_reference_genome_R64-1-1_20110203/s.cerevisiae-R64-1-1' + '\n')
out.close

In [ ]:
# Create input file for alignment to the YPS1009 reference genome.
outputPath = parentDir + 'alignments/YPS1009/'
with open('refAlignmentYPS1009_input.txt', 'w') as out:
    for inFastq in glob.glob(parentDir + 'alignments/*.fastq'):
        sampleName = re.sub('.fastq', '.sam', os.path.basename(inFastq))
        out.write(inFastq + ' ' + outputPath + sampleName + ' /home/glbrc.org/mplace/data/reference/YPS1009/YPS1009' + ' \n')
out.close

### Write the condor submit files for running bowtie2 

In [ ]:
# Step 5) Align reads to reference genomes S288C and YPS1009
# align reads to S288C using bowtie2 
# write the bowtie2 condor submit file
#S288C
with open('s288cbowtie2.submit', 'w') as submit:
    submit.write( "Universe                 = vanilla\n" )
    submit.write( "Executable               = runBowtie2.sh\n")
    submit.write( "Arguments                = $(fastqFile) $(sam) $(ref)\n")
    submit.write( "Error                    = s288c_bowtie2.submit.err\n")
    submit.write( "Log                      = s288c_bowtie2.submit.log\n")  
    submit.write( "Requirements             = OpSysandVer == \"CentOS7\"\n")
    submit.write( "Queue fastqFile, sam, ref from refAlignmentS288C_input.txt\n" )
submit.close()
#YPS1009
with open('yps1009bowtie2.submit', 'w') as submit:
    submit.write( "Universe                 = vanilla\n" )
    submit.write( "Executable               = runBowtie2.sh\n")
    submit.write( "Arguments                = $(fastqFile) $(sam) $(ref)\n")
    submit.write( "Error                    = yps1009_bowtie2.submit.err\n")
    submit.write( "Log                      = yps1009_bowtie2.submit.log\n")  
    submit.write( "Requirements             = OpSysandVer == \"CentOS7\"\n")
    submit.write( "Queue fastqFile, sam, ref from refAlignmentYPS1009_input.txt\n" )
submit.close()

# write shell script to run bowtie2
with open('runBowtie2.sh', 'w') as out:
    out.write("#!/bin/bash\n")
    out.write("source /opt/bifxapps/miniconda3/etc/profile.d/conda.sh\n")
    out.write("unset PYTHONPATH\n")  
    out.write("conda activate /home/glbrc.org/mplace/.conda/envs/riboSeq\n")
    out.write(f"bowtie2 -p 8 --phred33 -N 1 -x $3 -U $1 -S $2\n")
    out.write("conda deactivate")
out.close()

os.chmod('runBowtie2.sh', 0o0777)

###  on scarcity-submit run the above condor files

condor_submit s288cbowtie2.submit

condor_submit yps1009bowtie2.submit

### Sort the alignment files for the next step.

In [ ]:
# samtools sort ANEU-LOG-30_S2_R1_001.sam > ANEU-LOG-30_S2_R1_001-sorted.sam
# samtools mpileup -f ~/data/reference/S288C_reference_genome_R64-1-1_20110203/S288C_reference_sequence_R64-1-1_20110203.fsa ANEU-LOG-30_S2_R1_001-sorted.sam > ANEU-LOG-30_pileup.txt
# Create input file for samtools sort 
inputFilePath = parentDir + 'alignments/YPS1009/'
print(inputFilePath)
with open('sort-YPS1009_input.txt', 'w') as out:
    for sam in glob.glob(inputFilePath + '*.sam'):
        sampleName = re.sub('.sam', '.bam', os.path.basename(sam))
        out.write(sam + ' ' + inputFilePath + sampleName + ' \n')
out.close

In [ ]:
inputFilePath = parentDir + 'alignments/S288C/'
print(inputFilePath)
with open('sort-S288C_input.txt', 'w') as out:
    for sam in glob.glob(inputFilePath + '*.sam'):
        sampleName = re.sub('.sam', '.bam', os.path.basename(sam))
        out.write(sam + ' ' + inputFilePath + sampleName + ' \n')
out.close

### Write the condor submit files for sorting the alignment files.

In [ ]:
# Sort the alignment output files S288C and YPS1009
with open('sort.submit', 'w') as submit:
    submit.write( "Universe                 = vanilla\n" )
    submit.write( "Executable               = runsort.sh\n")
    submit.write( "Arguments                = $(sam) $(bam)\n")
    submit.write( "Error                    = sort.submit.err\n")
    submit.write( "Log                      = sort.submit.log\n")  
    submit.write( "Requirements             = OpSysandVer == \"CentOS7\"\n")
    #submit.write( "Queue sam, bam from  sort-S288C_input.txt\n" )
    submit.write( "Queue sam, bam from sort-YPS1009_input.txt\n" )
submit.close()

# write shell script to run sorting
with open('runsort.sh', 'w') as out:
    out.write("#!/bin/bash\n")
    out.write("source /opt/bifxapps/miniconda3/etc/profile.d/conda.sh\n")
    out.write("unset PYTHONPATH\n")  
    out.write("conda activate /home/glbrc.org/mplace/.conda/envs/riboSeq\n")
    out.write("samtools sort $1  > $2\n")
    out.write("conda deactivate")
out.close()

os.chmod('runsort.sh', 0o0777)

### Create a dictionary of S288C chromosome sizes

In [4]:
# S288C chromosome sizes
S288C_chromSizes = {}
with open('/mnt/bigdata/linuxhome/mplace/data/reference/S288C_reference_genome_R64-1-1_20110203/individualChroms/S288C.chromsizes', 'r') as f:
    for ln in f:
        chrom, length = ln.rstrip().split()
        if chrom not in S288C_chromSizes and chrom != '2-micron':
            S288C_chromSizes[chrom] = int(length)

print(S288C_chromSizes)

{'ref|NC_001133|': 230218, 'ref|NC_001134|': 813184, 'ref|NC_001135|': 316620, 'ref|NC_001136|': 1531933, 'ref|NC_001137|': 576874, 'ref|NC_001138|': 270161, 'ref|NC_001139|': 1090940, 'ref|NC_001140|': 562643, 'ref|NC_001141|': 439888, 'ref|NC_001142|': 745751, 'ref|NC_001143|': 666816, 'ref|NC_001144|': 1078177, 'ref|NC_001145|': 924431, 'ref|NC_001146|': 784333, 'ref|NC_001147|': 1091291, 'ref|NC_001148|': 948066, 'ref|NC_001224|': 85779}


In [ ]:
# Create information dictionaries for S288C
gff = '/mnt/bigdata/linuxhome/mplace/data/reference/S288C_reference_genome_R64-1-1_20110203/saccharomyces_cerevisiae_R64-1-1_20110208_noFasta.gff'
geneLookUp = {}    # dictionary of dictionaries key = chrom : {key = the -72 position of gene value = gene name }
gffDict = {}       # dictionary of dictionaries key = chrom : {key = {start : 0, end : 0, strand : '+'}

with open(gff, 'r') as g:
    for line in g:
        if line.startswith('#'):   # skip comment rows
            continue
        else:
            dat = line.split('\t')
            chrom = dat[0]
            if chrom not in S288C_chromSizes:
                continue
            if chrom not in geneLookUp:
                geneLookUp[chrom] = {}
            # add chrom to gffDict
            if chrom not in gffDict:
                gffDict[chrom] = {}
            # identify genes
            if dat[2] == 'gene':
                geneName = re.sub('ID=', '', dat[8].split(';')[0])
                # identify strand
                if dat[6] == '+':          # POSIIVE STRAND
                    if int(dat[3]) < 72:          
                        start = 0
                    else:
                        start = int(dat[3]) - 72
                    if start not in geneLookUp:
                        geneLookUp[chrom][start] = geneName
                    else:
                        print('Duplicate start ', geneName, geneLookUp[chrom][start])

                    if not int(dat[4]) + 60 > S288C_chromSizes[chrom]:
                        end   = int(dat[4]) + 60
                    else:
                        end = S288C_chromSizes[chrom]

                    # add final gene info for positive strand 
                    if geneName not in gffDict[chrom]:
                        gffDict[chrom][geneName] = {'strand' : '+', 'start': start, 'end': end }
                else:                      # NEGATIVE STRAND
                    if  int(dat[3]) - 60 < 60:
                        start = 0
                    else:
                        start = int(dat[3]) - 60
                    if start not in geneLookUp:
                        geneLookUp[chrom][start] = geneName
                    else:
                        print('Duplicate start minus strand ', geneName, geneLookUp[chrom][start]) 

                    if not int(dat[4]) + 72 > S288C_chromSizes[chrom]:
                        end = int(dat[4]) + 72
                    else:
                        end = S288C_chromSizes[chrom]
                    # add final gene info for positive strand 
                    if geneName not in gffDict[chrom]:
                        gffDict[chrom][geneName] = {'strand' : '-', 'start': start, 'end': end }


# write bed file for use with Samtools  -l flag, start and end positions have been adjust -72 from start and + 60 to end
# track name=yeast_genes description="All S.cerevisiae Gene locations"
with open('S288C-Genes.bed', 'w') as out:
    out.write('track name=yeast_genes description="All S.cerevisiae Gene locations"\n')
    for chrom in gffDict.keys():
        for gene, dat in gffDict[chrom].items():
            out.write(f'{chrom} {gffDict[chrom][gene]["start"]} {gffDict[chrom][gene]["end"]} {gene} {gffDict[chrom][gene]["strand"]}\n')

In [ ]:
# create a dictionary for bed file
genes = {}
with open('S288C-Genes.bed', 'r') as f:
    f.readline()
    for ln in f:
        dat = ln.rstrip().split()
        if dat[3] not in genes:
            genes[dat[3]] = [ dat[1], dat[2], dat[4]]

### Read sample SAM files and produce a count of read start positions that fall within -72 bases upstream of gene start and +60 of gene end.
#### I have created a modified bed file with the start and end positions reflecting the -72 and + 60 windows.

For example

New (plus strand):   ref|NC_001133| 263 709 YAL069W +       ,     Old (from GFF): ref|NC_001133|	335	649		+

New (minus strand):  ref|NC_001133| 1747 2241 YAL068C -     ,     Old (from GFF): ref|NC_001133|	1807	2169	-

In [ ]:
# YPS chromosome sizes, Need these values for the interval tree
YPS1009_chromSizes = {}
with open('/mnt/bigdata/linuxhome/mplace/data/reference/YPS1009/YPS1009.chromsizes', 'r') as f:
    for ln in f:
        chrom, length = ln.rstrip().split()
        if chrom not in YPS1009_chromSizes:
            YPS1009_chromSizes[chrom] = int(length)

print(YPS1009_chromSizes)

In [ ]:
# Create information dictionaries for YPS1009
gff = '/mnt/bigdata/linuxhome/mplace/data/reference/YPS1009/YPS1009.gff'
geneLookUp = {}    # dictionary of dictionaries key = chrom : {key = the -72 position of gene value = gene name }
gffDict = {}       # dictionary of dictionaries key = chrom : {key = {start : 0, end : 0, strand : '+'}

with open(gff, 'r') as g:
    for line in g:
        if line.startswith('#'):   # skip comment rows
            continue
        else:
            dat = line.split('\t')
            chrom = dat[0]
            if chrom not in YPS1009_chromSizes:
                continue
            if chrom not in geneLookUp:
                geneLookUp[chrom] = {}
            # add chrom to gffDict
            if chrom not in gffDict:
                gffDict[chrom] = {}
            # identify genes
            if dat[2] == 'gene':
                geneName = re.sub('ID=', '', dat[8].split(';')[0])
                # identify strand
                if dat[6] == '+':          # POSIIVE STRAND
                    if int(dat[3]) < 72:          
                        start = 0
                    else:
                        start = int(dat[3]) - 72
                    if start not in geneLookUp:
                        geneLookUp[chrom][start] = geneName
                    else:
                        print('Duplicate start ', geneName, geneLookUp[chrom][start])

                    if not int(dat[4]) + 60 > YPS1009_chromSizes[chrom]:
                        end   = int(dat[4]) + 60
                    else:
                        end = YPS1009_chromSizes[chrom]

                    # add final gene info for positive strand 
                    if geneName not in gffDict[chrom]:
                        gffDict[chrom][geneName] = {'strand' : '+', 'start': start, 'end': end }
                else:                      # NEGATIVE STRAND
                    if  int(dat[3]) - 60 < 60:
                        start = 0
                    else:
                        start = int(dat[3]) - 60
                    if start not in geneLookUp:
                        geneLookUp[chrom][start] = geneName
                    else:
                        print('Duplicate start minus strand ', geneName, geneLookUp[chrom][start]) 

                    if not int(dat[4]) + 72 > YPS1009_chromSizes[chrom]:
                        end = int(dat[4]) + 72
                    else:
                        end = YPS1009_chromSizes[chrom]
                    # add final gene info for positive strand 
                    if geneName not in gffDict[chrom]:
                        gffDict[chrom][geneName] = {'strand' : '-', 'start': start, 'end': end }


# write bed file for use with Samtools  -l flag, start and end positions have been adjust -72 from start and + 60 to end
# track name=yeast_genes description="All S.cerevisiae Gene locations"
with open('YPS1009-Genes.bed', 'w') as out:
    out.write('track name=yeast_genes description="All YPS1009 S.cerevisiae Gene locations"\n')
    for chrom in gffDict.keys():
        for gene, dat in gffDict[chrom].items():
            out.write(f'{chrom} {gffDict[chrom][gene]["start"]} {gffDict[chrom][gene]["end"]} {gene} {gffDict[chrom][gene]["strand"]}\n')

### Setup interval tree use with S288C.bed file for search the start positions of reads

Interval Tree implementation :  https://github.com/moonso/interval_tree

from interval_tree import IntervalTree
features = [

        [20,400,'id01'], 

        [30,400,'id02'], 

        [500,700,'id03'], 

        [1020,2400,'id04'], 

        [35891,29949,'id05'], 

        [899999,900000,'id06'], 
        
        [999000,999000,'id07']
    ]
my_tree = IntervalTree(features, 1, 1000000)

print('Ranges between 0 and 10: %s' % my_tree.find_range([0, 10]))
>Ranges between 0 and 10: []


In [5]:
from interval_tree import IntervalTree

prevChrom = 'ref|NC_001133|'
chromBeds = {}
with open('S288C-Genes.bed', 'r') as f:
    f.readline()                        # skip header
    features = []

    for ln in f:
        dat = ln.split()                #  ref|NC_001133| 263 709 YAL069W +

        if dat[0] == prevChrom:
            row = [int(dat[1]), int(dat[2]), dat[3]]
            features.append(row)
        elif dat[0] != prevChrom:
            chromBeds[prevChrom] = IntervalTree(features,1, int(S288C_chromSizes[prevChrom])) # write most recent
            if dat[0] not in chromBeds:
                chromBeds[dat[0]] = None
            features = []
            row = [int(dat[1]), int(dat[2]), dat[3]]
            features.append(row)
            prevChrom = dat[0]
    
    chromBeds[prevChrom] = IntervalTree(features,1, int(S288C_chromSizes[prevChrom])) # write most recent



#chrI = IntervalTree(features, 1, 230218)
# ref|NC_001147| 1089955 1090569 YOR396C-A -
#print(chromBeds['ref|NC_001133|'].find_range([120153, 120153]))

In [6]:
# load the S288C modified bed file
S288CBED = {}
with open('S288C-Genes.bed','r') as f:
    f.readline()
    for ln in f:
        dat = ln.rstrip().split()
        if dat[0] not in S288CBED:
            S288CBED[dat[0]] = {}
            if dat[3] not in S288CBED[dat[0]]:
                S288CBED[dat[0]][dat[3]] = {'start': int(dat[1]), 'end': int(dat[2]), 'strand':dat[4]}
        else:
            if dat[3] not in S288CBED[dat[0]]:
                S288CBED[dat[0]][dat[3]] = {'start': int(dat[1]), 'end': int(dat[2]), 'strand':dat[4]}

In [ ]:
S288CBED

### Count read start positions across All yeast genes

SAM field information:

 QNAME String [!-?A-~]{1,254} Query template NAME

 FLAG Int [0, 216 − 1] bitwise FLAG

 RNAME String \*|[:rname:∧*=][:rname:]* Reference sequence NAME11

 POS Int [0, 231 − 1] 1-based leftmost mapping POSition

 MAPQ Int [0, 28 − 1] MAPping Quality

 CIGAR String \*|([0-9]+[MIDNSHPX=])+ CIGAR string

 RNEXT String \*|=|[:rname:∧*=][:rname:]* Reference name of the mate/next read

 PNEXT Int [0, 231 − 1] Position of the mate/next read

 TLEN Int [−231 + 1, 231 − 1] observed Template LENgth

 SEQ String \*|[A-Za-z=.]+ segment SEQuence

 QUAL String [!-~]+ ASCII of Phred-scaled base QUALity+33

### MINUS STRAND

In [28]:
# **************************************   MINUS STRAND

#for aln in glob.glob('alignments/S288C/*-sorted.sam'):
    
#    outName = re.sub('-sorted.sam', '-RiboSeq-counts.txt', aln)
#    print('processing ', aln, outName)
    
# set up results dictionary for the counting of start positions
results = {}
for chromName in chromBeds.keys():
    if chromName not in results:
        results[chromName] = {}   

# open sam file , MAKE SURE THE SAM FILE IS SORTED (samtools sort)
# we are interested in columns  2, 3, 4 (FLAG, RNAME, POS)
# we use the NM tag to decide if there is an exact match
# NM Edit distance to the reference, so NM:i:0  is an exact match anything else is not.
#with open(aln, 'r') as f:
with open('alignments/S288C/ANEU-LOG-30-sorted.sam', 'r') as f:
    for line in f:
        # skip header information
        if not line.startswith('@'):
            read = line.split('\t')
            if read[1] == '4':             # these reads are unmapped
                continue
            elif read[1] == '16':           # mapped in forward direction
                chrom    = read[2]
                readStart = int(read[3])
                tags     = read[11:]       # section contains NM tag
                for tag in tags:           # find NM tag
                    if tag.startswith('NM:'):
                        myTag = tag.split('NM:i:')[1]   
                        if myTag == '0':
                            gene = chromBeds[chrom].find_range([readStart, readStart])   # which gene does this read start in
                            if gene is not None:                                       
                                for g in gene:                                         # this is a list, make sure we have the proper strand 
                                    if S288CBED[chrom][g]['strand'] == '-':
                                        #out.write(f'{chrom},  {readStart}, {tag}, {myTag}, {g}\n')
                                        #print( 'start ' , S288CBED[chrom][g]['start'], 'end', S288CBED[chrom][g]['end'],
                                        #                    'strand', S288CBED[chrom][g]['strand'], 'pos', readStart, 'cnts', 1 )
                                        if g not in results[chrom]:
                                            results[chrom][g] = {'start' : S288CBED[chrom][g]['start'], 'end': S288CBED[chrom][g]['end'],
                                                            'strand': S288CBED[chrom][g]['strand'], 'pos' : {readStart : 1 }}
                                        else:
                                            if readStart in results[chrom][g]['pos']:
                                                results[chrom][g]['pos'][readStart] += 1 
                                            else:
                                                results[chrom][g]['pos'][readStart] = 1     
                                                                    


In [ ]:
results['ref|NC_001134|']['YBL039C']

In [34]:
### Write table of start position counts

#with open(outName, 'w') as out:
with open('MINUS-TEST.txt', 'w') as out:
    for c in results.keys():
        for gene in results[c].keys():
            sortedPos = list(results[c][gene]['pos'].keys())
            startPos = results[c][gene]['start']
            endPos   = results[c][gene]['end']
            geneLength = endPos = startPos
            strand   = results[c][gene]['strand']
            sortedPos.sort()
            #print(c, gene, results[c][gene]['start'], results[c][gene]['end'], results[c][gene]['strand'] )
            outCodon = []         
            outPos = []           # genome position
            outCnt = []
              
            for p in range(results[c][gene]['start'], results[c][gene]['end']):
                outPos.append(str(p))
                if p in results[c][gene]['pos']:
                    outCnt.append(str(results[c][gene]['pos'][p]))
                else:
                    outCnt.append('0')
            
            # create the codon positions starting at -71
            for i, tmp in enumerate(outPos, start=-71):
                outCodon.append(str(i))
            
            out.write(f'{c}\n')
            outPos.reverse()
            posLine   = f'Genome_Position ' + ' '.join(outPos) + '\n'
            out.write(posLine)
            
            codonLine = f'{results[c][gene]["strand"]} ' + ' '.join(outCodon) + '\n'
            out.write(codonLine)
            
            outCnt.reverse()
            countLine = f'{gene} ' + ' '.join(outCnt)   + '\n'
            out.write(countLine)
            


### PLUS STRAND NEXT

In [ ]:
# PLUS STRAND

for aln in glob.glob('alignments/S288C/*-sorted.sam'):
    
    outName = re.sub('-sorted.sam', '-RiboSeq-counts.txt', aln)
    print('processing ', aln, outName)
    
    # set up results dictionary for the counting of start positions
    results = {}
    for chromName in chromBeds.keys():
        if chromName not in results:
            results[chromName] = {}   

    # open sam file , MAKE SURE THE SAM FILE IS SORTED (samtools sort)
    # we are interested in columns  2, 3, 4 (FLAG, RNAME, POS)
    # we use the NM tag to decide if there is an exact match
    # NM Edit distance to the reference, so NM:i:0  is an exact match anything else is not.
    with open(aln, 'r') as f:
        for line in f:
            # skip header information
            if not line.startswith('@'):
                read = line.split('\t')
                if read[1] == '4':             # these reads are unmapped
                    continue
                elif read[1] == '0':           # mapped in forward direction
                    chrom    = read[2]
                    readStart = int(read[3])
                    tags     = read[11:]       # section contains NM tag
                    for tag in tags:           # find NM tag
                        if tag.startswith('NM:'):
                            myTag = tag.split('NM:i:')[1]   
                            if myTag == '0':
                                gene = chromBeds[chrom].find_range([readStart, readStart])   # which gene does this read start in
                                if gene is not None:                                       
                                    for g in gene:                                         # this is a list, make sure we have the proper strand 
                                        if S288CBED[chrom][g]['strand'] == '+':
                                            #out.write(f'{chrom},  {readStart}, {tag}, {myTag}, {g}\n')
                                            #print( 'start ' , S288CBED[chrom][g]['start'], 'end', S288CBED[chrom][g]['end'],
                                            #                    'strand', S288CBED[chrom][g]['strand'], 'pos', readStart, 'cnts', 1 )
                                            if g not in results[chrom]:
                                                results[chrom][g] = {'start' : S288CBED[chrom][g]['start'], 'end': S288CBED[chrom][g]['end'],
                                                                'strand': S288CBED[chrom][g]['strand'], 'pos' : {readStart : 1 }}
                                            else:
                                                if readStart in results[chrom][g]['pos']:
                                                    results[chrom][g]['pos'][readStart] += 1 
                                                else:
                                                    results[chrom][g]['pos'][readStart] = 1     
                                                                        

### Write table of start position counts

    with open(outName, 'w') as out:
        for c in results.keys():
            for gene in results[c].keys():
                sortedPos = list(results[c][gene]['pos'].keys())
                sortedPos.sort()
                #print(c, gene, results[c][gene]['start'], results[c][gene]['end'], results[c][gene]['strand'] )
                outCodon = []
                outPos = []
                outCnt = []
                
                codonPos = -72  
                for p in range(results[c][gene]['start'], results[c][gene]['end']):
                    outCodon.append(str(codonPos))
                    codonPos +=1
                    outPos.append(str(p))
                    if p in results[c][gene]['pos']:
                        outCnt.append(str(results[c][gene]['pos'][p]))
                    else:
                        outCnt.append('0')
                    
                out.write(f'{c}\n')
                posLine   = f'Genome_Position ' + ' '.join(outPos) + '\n'
                out.write(posLine)
                codonLine = f'{results[c][gene]["strand"]} ' + ' '.join(outCodon) + '\n'
                out.write(codonLine)
                countLine = f'{gene} ' + ' '.join(outCnt)   + '\n'
                out.write(countLine)
                


### Samtools mpileup -- Previously run, but not really what was required.

In [ ]:
# samtools mpileup

#samtools mpileup -l regions.bed -f ~/data/reference/S288C_reference_genome_R64-1-1_20110203/S288C_reference_sequence_R64-1-1_20110203.fsa /
# ANEU-LOG-30_S2_R1_001-sorted.sam > ANEU-LOG-30_pileup.txt
# I created a bed file for all gene positions in YPS1009 and S288C and used this with pileup, this will only output the regions we are interested in viewing.


# create pileup output directories
pileupPath = parentDir + 'pileup/'
os.mkdir(pileupPath)
os.mkdir(pileupPath + 'YPS1009')
os.mkdir(pileupPath + 'S288C')


# Create samtools pileup input file for YPS1009
yps1009Ref = '/home/glbrc.org/mplace/data/reference/YPS1009/YPS1009.fasta'
tmpPath = pileupPath + 'YPS1009/'
with open('pileUp_YPS1009_input.txt', 'w') as out:
    for sam in glob.glob(parentDir + 'alignments/YPS1009/*.bam'):
        sampleName = re.sub('.bam', '-pileup.txt', os.path.basename(sam))
        out.write(sam + ' ' + tmpPath + sampleName + ' ' + yps1009Ref + '\n')
out.close


# Create samtools pileup input file for YPS1009
yps1009Ref = '/home/glbrc.org/mplace/data/reference/YPS1009/YPS1009.fasta'
tmpPath = pileupPath + 'YPS1009/'
with open('pileUp_YPS1009_input.txt', 'w') as out:
    for sam in glob.glob(parentDir + 'alignments/YPS1009/*.bam'):
        sampleName = re.sub('.bam', '-pileup.txt', os.path.basename(sam))
        out.write(sam + ' ' + tmpPath + sampleName + ' ' + yps1009Ref + '\n')
out.close


# Create pileup.sh input file for S288C
s288cRef = '/mnt/bigdata/linuxhome/mplace/data/reference/S288C_reference_genome_R64-1-1_20110203/S288C_reference_sequence_R64-1-1_20110203.fsa'
tmpPath = pileupPath + 'S288C/'
with open('pileUp_S288C_input.txt', 'w') as out:
    for bam in glob.glob(parentDir + 'alignments/S288C/*.bam'):
        sampleName = re.sub('.bam', '-pileup.txt', os.path.basename(bam))
        out.write(bam + ' ' + tmpPath + sampleName + ' ' + s288cRef + '\n')
out.close
# Setup pileup.sh job for S288C
with open('s288cPileUp.submit', 'w') as submit:
    submit.write( "Universe                 = vanilla\n" )
    submit.write( "Executable               = runS288CPileUp.sh\n")
    submit.write( "Arguments                = $(bam) $(pileup) $(ref)\n")
    submit.write( "Error                    = s288c_pileup.submit.err\n")
    submit.write( "Log                      = s288c_pileup.submit.log\n")  
    submit.write( "Requirements             = OpSysandVer == \"CentOS7\"\n")
    submit.write( "Queue bam, pileup, ref from pileUp_S288C_input.txt\n" )
submit.close()

# write shell script to run pileup
with open('runS288CPileUp.sh', 'w') as out:
    out.write("#!/bin/bash\n")
    out.write("source /opt/bifxapps/miniconda3/etc/profile.d/conda.sh\n")
    out.write("unset PYTHONPATH\n")  
    out.write("conda activate /home/glbrc.org/mplace/.conda/envs/riboSeq\n")
    out.write(f"samtools mpileup -f $3 $1 > $2\n")
    out.write("conda deactivate")
out.close()

os.chmod('runS288CPileUp.sh', 0o0777)

#YPS1009
with open('yps1009PileUp.submit', 'w') as submit:
    submit.write( "Universe                 = vanilla\n" )
    submit.write( "Executable               = runPileUp.sh\n")
    submit.write( "Arguments                = $(bam) $(pileup) $(ref) $\n")
    submit.write( "Error                    = yps1009_pileup.submit.err\n")
    submit.write( "Log                      = yps1009_pileup.submit.log\n")  
    submit.write( "Requirements             = OpSysandVer == \"CentOS7\"\n")
    submit.write( "Queue bam, pileup, ref from pileUp_YPS1009_input.txt\n" )
submit.close()

# write shell script to run cutadapt
with open('runPileUp.sh', 'w') as out:
    out.write("#!/bin/bash\n")
    out.write("source /opt/bifxapps/miniconda3/etc/profile.d/conda.sh\n")
    out.write("unset PYTHONPATH\n")  
    out.write("conda activate /home/glbrc.org/mplace/.conda/envs/riboSeq\n")
    out.write(f"samtools mpileup -f $3 $1 > $2\n")
    out.write("conda deactivate")
out.close()

os.chmod('runPileUp.sh', 0o0777)

# open pileup file and create table
sample = {}   # key = chrom : geneName { positions: [],  Counts : []}

with open('pileup/S288C/ANEU-LOG-30_S2_R1_001-pileup.txt', 'r') as f:
    for ln in f:
        dat = ln.split('\t')
        if dat[0] not in sample:            # we encounter a new chromosome
            gene = chromBeds[dat[0]].find_range([int(dat[1]), int(dat[1])])
            sample[dat[0]] = {}
            for gn in gene:                  # new genes added
                sample[dat[0]][gn] = {'start' : genes[gn][0], 'end': genes[gn][1], 'strand':genes[gn][2], 'pos' : [dat[1]], 'cnts':[dat[3]] }
            print(dat[0], gene,dat[1])
        else:
            gene = chromBeds[dat[0]].find_range([int(dat[1]), int(dat[1])])      # find gene

            for gn in gene:
                if gn not in sample[dat[0]]:
                    sample[dat[0]][gn] = {'start' : genes[gn][0], 'end': genes[gn][1], 'strand':genes[gn][2], 'pos' : [dat[1]], 'cnts':[dat[3]] }
                else:
                    sample[dat[0]][gn]['pos'].append(dat[1])
                    sample[dat[0]][gn]['cnts'].append(dat[3])